<a href="https://colab.research.google.com/github/stevejj4/SHOFCO-SUN/blob/master/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import auth #Authorizing google colab
auth.authenticate_user()
import gspread #credentials for google sheet
from google.auth import default
creds,_ = default()
gc = gspread.authorize(creds) #Authorizing the connection
import pandas as pd
worksheet = gc.open('Nyando').sheet1 #defining the worksheet
rows = worksheet.get_all_values() #getting all values in list of row
df = pd.DataFrame(rows)
df.columns = df.iloc[0] #creating columns name
df = df.iloc[1:]

In [8]:
Facilitator_check = df.groupby(['1. Group Name:', '2. Facilitator Name', '7. Ward']).size()
# Convert the result to a DataFrame and reset the index
Facilitator_summary = Facilitator_check.reset_index(name='Count')
print(Facilitator_summary)

    1. Group Name: 2. Facilitator Name            7. Ward  Count
0          ACHAR A        David Maseno              Ahero     21
1          ACHAR B        David Maseno              Ahero     13
2    ACHAR WOMEN C        David Maseno              Ahero      3
3         ACHEGO B      Pamella Atieno          East Kano     18
4         ACHEGO C      Pamella Atieno          East Kano     16
..             ...                 ...                ...    ...
765  Young Wawidhi        Mercy Sabina          East Kano     11
766         ZAWADI         Grace Ohuru             Kobura      4
767         ZAWADI         Sophy Akoth             Kobura      3
768   ZAWADI GROUP         Grace Ohuru  Kabonyo/Kanyagwal      8
769    ZEBRA GROUP         Sophy Akoth             Kobura      8

[770 rows x 4 columns]


In [9]:
import pandas as pd

def check_group_wards(dataframe, group_cols, ward_col):
    """
    Check if groups belong to only one ward.

    Parameters:
    - dataframe (pd.DataFrame): The input DataFrame.
    - group_cols (list): List of column names to group by (e.g., group name and facilitator).
    - ward_col (str): The column name representing wards.

    Returns:
    - pd.DataFrame: Summary with unique ward counts for each group.
    - pd.DataFrame: Groups belonging to more than one ward.
    - pd.DataFrame: Groups belonging to exactly one ward.
    """
    # Group and count unique wards
    ward_check = dataframe.groupby(group_cols)[ward_col].nunique().reset_index()
    ward_check.rename(columns={ward_col: 'Unique Wards'}, inplace=True)

    # Groups with more than one ward
    multiple_wards = ward_check[ward_check['Unique Wards'] > 1]

    # Groups with exactly one ward
    single_ward = ward_check[ward_check['Unique Wards'] == 1]

    return ward_check, multiple_wards, single_ward

# Example usage:
# Assuming your DataFrame is named `df`
group_cols = ['1. Group Name:', '2. Facilitator Name']
ward_col = '7. Ward'

# Call the function
ward_summary, multiple_wards, single_wards = check_group_wards(df, group_cols, ward_col)

# Display results
print("Ward Summary:")
print(ward_summary)

print("\nGroups belonging to more than one ward:")
print(multiple_wards)

print("\nGroups belonging to exactly one ward:")
print(single_wards)


Ward Summary:
    1. Group Name: 2. Facilitator Name  Unique Wards
0          ACHAR A        David Maseno             1
1          ACHAR B        David Maseno             1
2    ACHAR WOMEN C        David Maseno             1
3         ACHEGO B      Pamella Atieno             1
4         ACHEGO C      Pamella Atieno             1
..             ...                 ...           ...
762  Young Wawidhi        Mercy Sabina             1
763         ZAWADI         Grace Ohuru             1
764         ZAWADI         Sophy Akoth             1
765   ZAWADI GROUP         Grace Ohuru             1
766    ZEBRA GROUP         Sophy Akoth             1

[767 rows x 3 columns]

Groups belonging to more than one ward:
    1. Group Name: 2. Facilitator Name  Unique Wards
432    NYARAKAKA A      Pamella Atieno             2
649     TIN OK CHA     Pamela Odhiambo             2
716  WACHO GI TIMO      Pamella Atieno             2

Groups belonging to exactly one ward:
    1. Group Name: 2. Facilitator 

In [10]:
def check_group_facilitators(dataframe, group_col, facilitator_col):
    """
    Check if groups belong to only one facilitator.

    Parameters:
    - dataframe (pd.DataFrame): The input DataFrame.
    - group_col (str): The column representing group names.
    - facilitator_col (str): The column representing facilitators.

    Returns:
    - pd.DataFrame: Summary with unique facilitator counts for each group.
    - pd.DataFrame: Groups belonging to more than one facilitator.
    - pd.DataFrame: Groups belonging to exactly one facilitator.
    """
    # Group and count unique facilitators
    facilitator_check = dataframe.groupby(group_col)[facilitator_col].nunique().reset_index()
    facilitator_check.rename(columns={facilitator_col: 'Unique Facilitators'}, inplace=True)

    # Groups with more than one facilitator
    multiple_facilitators = facilitator_check[facilitator_check['Unique Facilitators'] > 1]

    # Groups with exactly one facilitator
    single_facilitator = facilitator_check[facilitator_check['Unique Facilitators'] == 1]

    return facilitator_check, multiple_facilitators, single_facilitator

# Example usage:
# Assuming your DataFrame is named `df`
group_col = '1. Group Name:'
facilitator_col = '2. Facilitator Name'

# Call the function
facilitator_summary, multiple_facilitators, single_facilitator = check_group_facilitators(df, group_col, facilitator_col)

# Display results
print("Facilitator Summary:")
print(facilitator_summary)

print("\nGroups belonging to more than one facilitator:")
print(multiple_facilitators)

print("\nGroups belonging to exactly one facilitator:")
print(single_facilitator)


Facilitator Summary:
    1. Group Name:  Unique Facilitators
0          ACHAR A                    1
1          ACHAR B                    1
2    ACHAR WOMEN C                    1
3         ACHEGO B                    1
4         ACHEGO C                    1
..             ...                  ...
554       Yie Geno                    4
555  Young Wawidhi                    1
556         ZAWADI                    2
557   ZAWADI GROUP                    1
558    ZEBRA GROUP                    1

[559 rows x 2 columns]

Groups belonging to more than one facilitator:
                 1. Group Name:  Unique Facilitators
6          ACHIEVERS NYAMKEBE A                    2
10   AHERO MOTORCYCLE MECHANICS                    2
17                AMANI KOBONGO                    2
18                AMANI SUPPORT                    3
19               AMANI UNITED A                    3
..                          ...                  ...
543        Wan Gi Yie Ni Timore                    2
547

In [11]:
def check_group_ward_facilitator(dataframe, group_col, ward_col, facilitator_col):
    """
    Check if groups belong to more than one ward and more than one facilitator.

    Parameters:
    - dataframe (pd.DataFrame): The input DataFrame.
    - group_col (str): The column representing group names.
    - ward_col (str): The column representing wards.
    - facilitator_col (str): The column representing facilitators.

    Returns:
    - pd.DataFrame: Summary with unique ward and facilitator counts for each group.
    - pd.DataFrame: Groups belonging to more than one ward and more than one facilitator.
    - pd.DataFrame: Groups belonging to exactly one ward and one facilitator.
    """
    # Group by and count unique wards and facilitators
    summary = dataframe.groupby(group_col).agg(
        Unique_Wards=(ward_col, 'nunique'),
        Unique_Facilitators=(facilitator_col, 'nunique')
    ).reset_index()

    # Groups with more than one ward and more than one facilitator
    multiple_wards_facilitators = summary[
        (summary['Unique_Wards'] > 1) & (summary['Unique_Facilitators'] > 1)
    ]

    # Groups with exactly one ward and one facilitator
    single_ward_facilitator = summary[
        (summary['Unique_Wards'] == 1) & (summary['Unique_Facilitators'] == 1)
    ]

    return summary, multiple_wards_facilitators, single_ward_facilitator

# Example usage:
# Assuming your DataFrame is named `df`
group_col = '1. Group Name:'
ward_col = '7. Ward'
facilitator_col = '2. Facilitator Name'

# Call the function
summary, multiple_wards_facilitators, single_ward_facilitator = check_group_ward_facilitator(
    df, group_col, ward_col, facilitator_col
)

# Display results
print("Group Summary:")
print(summary)

print("\nGroups belonging to more than one ward and more than one facilitator:")
print(multiple_wards_facilitators)

print("\nGroups belonging to exactly one ward and one facilitator:")
print(single_ward_facilitator)


Group Summary:
    1. Group Name:  Unique_Wards  Unique_Facilitators
0          ACHAR A             1                    1
1          ACHAR B             1                    1
2    ACHAR WOMEN C             1                    1
3         ACHEGO B             1                    1
4         ACHEGO C             1                    1
..             ...           ...                  ...
554       Yie Geno             1                    4
555  Young Wawidhi             1                    1
556         ZAWADI             1                    2
557   ZAWADI GROUP             1                    1
558    ZEBRA GROUP             1                    1

[559 rows x 3 columns]

Groups belonging to more than one ward and more than one facilitator:
                 1. Group Name:  Unique_Wards  Unique_Facilitators
10   AHERO MOTORCYCLE MECHANICS             2                    2
63                  Bet Ok Miyi             2                    2
267   Laja Kochieng Women Group          

In [12]:
# download facilitator_check
Facilitator_summary.to_csv('Facilitator_check.csv', index=False)
from google.colab import files
files.download('Facilitator_check.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>